In [54]:
from dash import Dash, html, dcc, Input, Output, callback, Patch, ALL, State

import dash_cytoscape as cyto
from matplotlib import style


app = Dash()

#Get the ID for the next node to be added to the list
#Goes through all ids and if they are integer finds the highest value and adds 1 to it, if none are integers returns 0. 
def getNextNodeID(elements):
    idList = []
    #print(elements)
    for element in elements:
        #print(element, ' element')
        if 'id' in element['data'].keys():
            if (element['data']['id'].isdigit()):
                idList.append(int(element['data']['id']))
    if len(idList) == 0:
        return 1
    else:
        return max(idList) + 1   
    
def addNewNode(elements):
    nodeDict = {}
    dataDict = {}
    nodeDict['id'] = str(getNextNodeID(elements))
    nodeDict['label'] = str(getNextNodeID(elements))
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default'
    #print(elements, ' addNode ', elements.append(dataDict))
    elements.append(dataDict)
    return elements

def removeNode(node, elements):
    #print('before remove ', elements)
    for element in elements:
      if 'id' in element['data'].keys():
        if node['id'] == element['data']['id']:
            elements.remove(element)
    #print('after remove ', elements)
    return elements

def removeEdge(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                elements.remove(element)
    return elements

def isEdgeInElements(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                return True
    return False

def addNewEdge(node1, node2, elements):
    edgeDict = {}
    dataDict = {}
    edgeDict['source'] = node1['id']
    edgeDict['target'] = node2['id']
    dataDict['data'] = edgeDict
    if isEdgeInElements(edgeDict, elements):
        return elements
    else:
        elements.append(dataDict)
        return elements

def setNodeColorToselected(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['classes'] = 'selected'
    return elements

def unselectNode(node, elements):
    for element in elements:
      if 'id' in element['data'].keys():
        if element['data']['id'] == node['id']:
            if 'classes' in element.keys():
                element['classes'] = 'default'
    return elements

app.layout = [
    dcc.Store(id='CurrentlySelectedNode'),
    dcc.Store(id='nodeSelectedForDeletion'),
    dcc.Store(id='selectedEdge'),
    dcc.Store(id='addEdgesFlag'),
    #graph entry
    html.Button(id="addEdges", children="Start Adding Edges"),
    html.Button(id='addNode_',children='Add a new Graph Node'),
    html.Button(id='RemoveNode', children='Remove a Specific Graph Node'),
    html.Button(id='RemoveEdge', children='Remove a Specific Graph Edge'),
    dcc.Store(id='Flags'),
    dcc.Store(id='edgeFlags'),
    html.H5(id="cytoText"),
    cyto.Cytoscape(
        id='cytoscape',
        elements=[
            {'data': {'id': 'escape', 'label': 'escape'}},
            {'data': {'id': '1', 'label': '1'}}, 
            {'data': {'id': '2', 'label': '2'}}, 
            {'data': {'id': '3', 'label': '3'}},
            {'data': {'source': '1', 'target': '2'}}, 
            {'data': {'source': '2', 'target': '3'}}
        ],
        #layout={'name': 'breadthfirst'},
        style={'width': '400px', 'height': '500px'},
        stylesheet=[
            {
                'selector': 'default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.selected',
                'style': {
                    'background-color': '#0099ff',
                    'content': 'data(label)'
                }
            }
        ]
    ),
]

#"Add Node" button callback    
@callback(
    Output("cytoscape", "elements",  allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("addNode_", "n_clicks"),
    prevent_initial_call=True,
)
def addNewNodeCallback(cyto, n_cl):
    return addNewNode(cyto)

#Callback for adding edges by tapping two nodes
@callback(
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    State("CurrentlySelectedNode", "data"),
    State("cytoscape", "elements"),
    State("RemoveNode", "children"),
    Input("cytoscape", "tapNodeData"),
    State("Flags", "data"),
    State("addEdgesFlag", "data"),
    State("nodeSelectedForDeletion", "data"),
    prevent_initial_call=True,
)
def tapNodeAddEdge(prevNode, elements, rmNodeButton, tappedNode, flags, addEdgesFlag, nodeSelectedForDeletion):
    if (addEdgesFlag == None) or (addEdgesFlag == False):
        store = None
        if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
            elements = removeNode(tappedNode, elements)
            flags['removeNode'] = False
            rmNodeButton = "Remove a Specific Graph Node"
            return flags, rmNodeButton, elements, store, nodeSelectedForDeletion
        elif nodeSelectedForDeletion == None:
            nodeSelectedForDeletion = tappedNode
            elements = setNodeColorToselected(tappedNode, elements)
            rmNodeButton = "Remove the Selected Graph Node"
        else:
            elements = unselectNode(tappedNode, elements)
            elements = unselectNode(nodeSelectedForDeletion, elements)
            nodeSelectedForDeletion = None
            rmNodeButton = "Remove a Specific Graph Node"
        return flags, rmNodeButton, elements, None, nodeSelectedForDeletion
    else:
        #print(elements)
        store = None
        #print(flags)
        if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
                elements = removeNode(tappedNode, elements)
                flags['removeNode'] = False
                rmNodeButton = "Remove a Specific Graph Node"
                return flags, rmNodeButton, elements, store
        else:
            if prevNode == None:
                store = tappedNode
                elements = setNodeColorToselected(tappedNode, elements)
                rmNodeButton = "Remove the Selected Graph Node"
            else:
                elements = unselectNode(prevNode, elements)
                elements = addNewEdge(prevNode, tappedNode, elements)
                elements = unselectNode(tappedNode, elements)
                store = None   
                rmNodeButton = "Remove a Specific Graph Node"
        return flags, rmNodeButton, elements, store, nodeSelectedForDeletion

#"Remove Node" buttton callback
@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    State("cytoscape", "elements"),
    State("Flags", "data"),
    State("CurrentlySelectedNode", "data"),
    Input("RemoveNode", "n_clicks"),
    State("RemoveNode", "children"),
    State("nodeSelectedForDeletion", "data"),
    prevent_initial_call=True,
    )
def removeNodePressed(elements, flags, selectedNode, clicks, rmNodeButton, nodeSelectedForDeletion):
    if nodeSelectedForDeletion != None:
        elements = removeNode(nodeSelectedForDeletion, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        nodeSelectedForDeletion = None
    elif selectedNode != None:
        elements = removeNode(selectedNode, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        selectedNode = None
    else:
        dictF = {}
        dictF['removeNode'] = True
        flags = dictF
        rmNodeButton = "Select Graph Node To Remove"
    return elements, flags, rmNodeButton, selectedNode, nodeSelectedForDeletion

#"Remove Edge" button callback
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    Input("RemoveEdge", "n_clicks"),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    State("selectedEdge", "data"),
    prevent_initial_call=True,
)
def removeEdgeButton(button, edgeFlags, elements, selectedEdge):
    if (edgeFlags == None) or (edgeFlags == False):
        if (selectedEdge == None):
            flag = True
            edgeFlags = flag
            buttonMSG = "Select Edge to be Removed"
        else:
            elements = removeEdge(selectedEdge, elements)
            selectedEdge = None
            buttonMSG = "Remove a Specific Graph Edge"
    else:
        edgeFlags = False
        buttonMSG = "Remove a Specific Graph Edge"
    return buttonMSG, elements, edgeFlags, selectedEdge

#Callback on tapped/clicked edge
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    Input("cytoscape", "tapEdgeData"),
    prevent_initial_call=True,
)
def selctedEdge(flags, elements, edge):
    selectedEdge = None
    if (flags != None) and (flags == True):
        elements = removeEdge(edge, elements)
        selectedEdge = None
        buttonMSG = "Remove a Specific Graph Edge"
        flags = False
    elif (flags == None) or (flags == False):
        selectedEdge = edge
        buttonMSG = "Remove Selected Edge"
    return buttonMSG, elements, flags, selectedEdge


#Button to start/stop adding edges
@callback(
    Output("addEdgesFlag", "data"),
    Output("addEdges", "children"),
    Input("addEdges", "n_clicks"),
    State("addEdgesFlag", "data"),
    prevent_initial_call=True,
)
def edgeAddButtonPressed(clicks, edgesToggle):
    if (edgesToggle==None) or (edgesToggle==False):
        edgesToggle = True
        buttonMSG = "Stop Adding Edges"
    elif(edgesToggle == True):
        edgesToggle = False
        buttonMSG = "Start Adding Edges"
    return edgesToggle, buttonMSG

    
        
app.run(debug=True)
#app.run(debug=True, host= "192.168.0.102", port=8052)
